In [1]:
%matplotlib inline


# Segmenting the picture of greek coins in regions

This example uses `spectral_clustering` on a graph created from
voxel-to-voxel difference on an image to break this image into multiple
partly-homogeneous regions.

This procedure (spectral clustering on an image) is an efficient
approximate solution for finding normalized graph cuts.

There are two options to assign labels:

* with 'kmeans' spectral clustering will cluster samples in the embedding space
  using a kmeans algorithm
* whereas 'discrete' will iteratively search for the closest partition
  space to the embedding space.


In [2]:
# Author: Gael Varoquaux <gael.varoquaux@normalesup.org>, Brian Cheung
# License: BSD 3 clause

import time

import numpy as np
from scipy.ndimage import gaussian_filter as scipy_gaussian_filter

import matplotlib.pyplot as plt
import skimage
from skimage.data import coins
from skimage.transform import rescale as skimage_rescale

from sklearn.feature_extraction import image

from sklearn.cluster import spectral_clustering
from sklearn.utils.fixes import parse_version

from scipy import sparse as scipy_sparse

# these were introduced in skimage-0.14
if parse_version(skimage.__version__) >= parse_version("0.14"):
    rescale_params = {"anti_aliasing": False}
else:
    rescale_params = {}

def create_image_graph(coins, gaussian_filter, rescale, return_as=scipy_sparse.coo_matrix):
    # Resize it to 20% of the original size to speed up the processing
    # Applying a Gaussian filter for smoothing prior to down-scaling
    # reduces aliasing artifacts.
    smoothened_coins = gaussian_filter(coins, sigma=2)
    # Taking the old array for cucim as we don't need to change cucim to support Array API
    # We'll change skimage to support Array API rather.    
    # smoothened_coins = smoothened_coins._array if hasattr(smoothened_coins, '_array') else smoothened_coins
    rescaled_coins = rescale(smoothened_coins, 0.2, mode="reflect", **rescale_params)

    # Convert the image into a graph with the value of the gradient on the
    # edges.

    graph = image.img_to_graph(rescaled_coins, return_as=return_as)
    return rescaled_coins, graph

## With NumPy

In [3]:
import numpy.array_api as npx

/tmp/ipykernel_29912/3403852876.py:1: UserWarning: The numpy.array_api submodule is still experimental. See NEP 47.
  import numpy.array_api as npx


In [4]:
%time
rescaled_coins1, graph1 = create_image_graph(npx.asarray(coins()), scipy_gaussian_filter, skimage_rescale)
rescaled_coins1, graph1

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 4.53 µs


/work/aktech/packages/scikit-image/skimage/transform/_warps.py:189: UserWarning: The numpy.array_api submodule is still experimental. See NEP 47.
  import cupy.array_api as cpx


(array([[0.49788643, 0.52164502, 0.50993125, ..., 0.38823529, 0.35625159,
         0.10562771],
        [0.50185893, 0.52164502, 0.51372549, ..., 0.38431373, 0.35240642,
         0.11347084],
        [0.50972753, 0.52156863, 0.51372549, ..., 0.38026483, 0.34856124,
         0.12521008],
        ...,
        [0.29419404, 0.26282149, 0.27830405, ..., 0.29816654, 0.22645786,
         0.03524319],
        [0.29416858, 0.2627451 , 0.27438248, ..., 0.29816654, 0.21077158,
         0.03132162],
        [0.29027247, 0.25889992, 0.27817673, ..., 0.29816654, 0.20285205,
         0.02742552]]),
 <23331x23331 sparse matrix of type '<class 'numpy.float64'>'
 	with 115895 stored elements in COOrdinate format>)

In [5]:
type(rescaled_coins1)

numpy.ndarray

## With CuPy

In [6]:
from cupyx.scipy.ndimage import gaussian_filter as cupy_gaussian_filter
from cupyx.scipy import sparse as cupy_sparse
# from cucim.skimage.transform import rescale as cucim_rescale
import cupy.array_api as cpx

In [7]:
%time
# rescale_function = cucim_rescale
rescale_function = skimage_rescale

orig_coins = cpx.asarray(coins())
rescaled_coins2, graph2 = create_image_graph(orig_coins, cupy_gaussian_filter, rescale_function, return_as=cupy_sparse.coo_matrix)
rescaled_coins2, graph2

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 3.58 µs


AxisError: source: axis 0 is out of bounds for array of dimension 0

#### This should return new Array when implementation is complete

In [ ]:
type(rescaled_coins2)

In [ ]:
vars(graph2)

## Plots: WIP

In [ ]:
# Take a decreasing function of the gradient: an exponential
# The smaller beta is, the more independent the segmentation is of the
# actual image. For beta=1, the segmentation is close to a voronoi
beta = 10
eps = 1e-6

def set_graph_data(array, graph):
    graph.data = array.exp(-beta * array.asarray(graph.data) / array.std(array.asarray(graph.data))) + eps
    graph.row = array.asarray(graph.row)
    graph.col = array.asarray(graph.col)
    # graph.data = graph.data._array
    return graph

graph1 = set_graph_data(npx, graph1)
graph2 = set_graph_data(cpx, graph2)
# Apply spectral clustering (this step goes much faster if you have pyamg
# installed)
N_REGIONS = 25

In [ ]:
vars(graph2)

Visualize the resulting regions



In [ ]:
labelling_methods = ["discretize"]

def plot_(graph, rescaled_coins, xp):
    for assign_labels in labelling_methods:
        t0 = time.time()
        labels = spectral_clustering(
            graph, n_clusters=N_REGIONS, assign_labels=assign_labels, random_state=42
        )
        t1 = time.time()
        labels = xp.reshape(labels, rescaled_coins.shape)
        plt.figure(figsize=(5, 5))
        if xp.__name__ == 'cupy.array_api':
            rescaled_coins = rescaled_coins._array.get()
            labels = labels._array.get()
        plt.imshow(rescaled_coins, cmap=plt.cm.gray)
        for l in range(N_REGIONS):
            plt.contour(labels == l, colors=[plt.cm.nipy_spectral(l / float(N_REGIONS))])
        plt.xticks(())
        plt.yticks(())
        title = "Spectral clustering: %s, %.2fs" % (assign_labels, (t1 - t0))
        print(title)
        plt.title(title)
    plt.show()

## Segmentation with NumPy Array

In [ ]:
plot_(graph1, rescaled_coins1, npx)

## Segmentation with CuPy Array

plot_(graph1, rescaled_coins1)

In [ ]:
plot_(graph2, rescaled_coins2, cpx)